<h2>Import dependencies</h2>

In [474]:
import pandas as pd
import numpy as np
import datetime # to handle date/time attributes
from os import listdir # os is a module for interacting with the OS
from os.path import isfile, join # to verify file object, and concatenate paths
import glob # to find pathnames matching a specific pattern
import re # regular expressions :)

# set seed for reproducibility
np.random.seed(0)

<h2>Import data</h2>

In [475]:
apps_df = pd.read_csv("./data/googleplaystore.csv")
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


<h2>Explore Data Types</h2>

In [476]:
print("Shape of data (rows,columns): ",apps_df.shape)
print(apps_df.dtypes)

Shape of data (rows,columns):  (10841, 13)
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


All of the columns are objects except for the rating column which is float. We will try to change this by changing some of the columns types to numeric ones, while others to strings.

<h3>Reviews</h3>

We check first if all of the values are actually numeric.

In [477]:
apps_df.Reviews.str.isnumeric().sum()

10840

There seems to be one value out of the 10841 values that is non-numeric.

In [478]:
apps_df[~apps_df.Reviews.str.isnumeric()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,11-Feb-18,1.0.19,4.0 and up,NaN


This app has a rating of 19.0, which doesn't make any sense. Also, its size, price and category attributes have weird values, therefore we should simply remove this entire row .

In [479]:
apps_df = apps_df[apps_df.Reviews.str.isnumeric()]

Finally, we convert the Reviews column's type to numeric

In [480]:
apps_df.Reviews=pd.to_numeric(apps_df.Reviews)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h3>Size</h3>

The size attribute should also be numeric. 

In [481]:
apps_df.Size.value_counts()

Varies with device    1695
11M                    198
12M                    196
14M                    194
13M                    191
                      ... 
157k                     1
485k                     1
232k                     1
696k                     1
924k                     1
Name: Size, Length: 461, dtype: int64

It seems that most of the values have either the suffix M or the suffix k. We could replace them by 10^6 and 10^3 respectively to make them numeric

In [482]:
apps_df.Size=apps_df.Size.str.replace('k','e+3')
apps_df.Size=apps_df.Size.str.replace('M','e+6')
apps_df.Size.head()

0     19e+6
1     14e+6
2    8.7e+6
3     25e+6
4    2.8e+6
Name: Size, dtype: object

We now make sure that all of the values are numeric before converting the entire column

In [483]:
def is_numeric(v):
    try:
        float(v)
        return True
    except ValueError:
        return False
    
temp=apps_df.Size.apply(lambda x: is_numeric(x))
apps_df.Size[~temp].value_counts()

Varies with device    1695
Name: Size, dtype: int64

There seems to be many rows that have size attributes with the values "Varies with device". We will simply change all of them to NaN.

In [484]:
apps_df.Size=apps_df.Size.replace('Varies with device',np.nan)

Finally, we convert the column's type to numeric

In [485]:
apps_df.Size=pd.to_numeric(apps_df.Size)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h3>Installs</h3>

We first check the unique values of this column.

In [486]:
apps_df.Installs.value_counts()

1,000,000+        1579
10,000,000+       1252
100,000+          1169
10,000+           1054
1,000+             907
5,000,000+         752
100+               719
500,000+           539
50,000+            479
5,000+             477
100,000,000+       409
10+                386
500+               330
50,000,000+        289
50+                205
5+                  82
500,000,000+        72
1+                  67
1,000,000,000+      58
0+                  14
0                    1
Name: Installs, dtype: int64

All of the values are either pure numbers or numbers prefixed with the sign '+'. We can convert the latter by simply removing the '+' sign.

In [487]:
apps_df.Installs=apps_df.Installs.apply(lambda x: x.strip('+'))
apps_df.Installs=apps_df.Installs.apply(lambda x: x.replace(',',''))
apps_df.Installs.value_counts()

1000000       1579
10000000      1252
100000        1169
10000         1054
1000           907
5000000        752
100            719
500000         539
50000          479
5000           477
100000000      409
10             386
500            330
50000000       289
50             205
5               82
500000000       72
1               67
1000000000      58
0               15
Name: Installs, dtype: int64

Now that everything seems to be in order, we can safetly convert this column's type to numeric.

In [488]:
apps_df.Installs=pd.to_numeric(apps_df.Installs)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs            int64
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h3>Price</h3>

We first get a feeling of the format of this column

In [489]:
apps_df.Price.value_counts()

0           10040
$0.99         148
$2.99         129
$1.99          73
$4.99          72
            ...  
$4.29           1
$379.99         1
$1.76           1
$3.02           1
$389.99         1
Name: Price, Length: 92, dtype: int64

All of the values are either 0 or are numbers prefixed with the dollar sign. We will begin by removing the dollar signs.

In [490]:
apps_df.Price=apps_df.Price.apply(lambda x: x.strip('$'))
apps_df.Price.value_counts()

0          10040
0.99         148
2.99         129
1.99          73
4.99          72
           ...  
200.00         1
25.99          1
2.95           1
3.90           1
4.29           1
Name: Price, Length: 92, dtype: int64

Finally, we make sure that all of the values are numeric then convert the column's dtype

In [491]:
temp=apps_df.Price.apply(lambda x: is_numeric(x))
apps_df.Price[~temp].value_counts()

Series([], Name: Price, dtype: int64)

In [492]:
apps_df.Price=pd.to_numeric(apps_df.Price)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs            int64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h2>Duplicates</h2>

Check if all app names are unique

In [493]:
apps_df["App"].is_unique

False

In [494]:
duplicates = apps_df[apps_df["App"].isin(apps_df[apps_df.duplicated()]["App"])]
duplicates.sort_values("App", inplace = False) 

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1393,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3800000.0,500000,Free,0.00,Everyone 10+,Health & Fitness,17-Feb-17,1.9,2.3.3 and up
1407,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3800000.0,500000,Free,0.00,Everyone 10+,Health & Fitness,17-Feb-17,1.9,2.3.3 and up
2543,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26000000.0,1000000,Free,0.00,Everyone,Medical,27-Jul-18,7.4.1,5.0 and up
2322,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26000000.0,1000000,Free,0.00,Everyone,Medical,27-Jul-18,7.4.1,5.0 and up
2256,2017 EMRA Antibiotic Guide,MEDICAL,4.4,12,3800000.0,1000,Paid,16.99,Everyone,Medical,27-Jan-17,1.0.5,4.0.3 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3014,"theScore: Live Sports Scores, News, Stats & Vi...",SPORTS,4.4,133825,34000000.0,10000000,Free,0.00,Everyone 10+,Sports,25-Jul-18,6.17.2,4.4 and up
3085,"theScore: Live Sports Scores, News, Stats & Vi...",SPORTS,4.4,133833,34000000.0,10000000,Free,0.00,Everyone 10+,Sports,25-Jul-18,6.17.2,4.4 and up
3118,trivago: Hotels & Travel,TRAVEL_AND_LOCAL,4.2,219848,NaN,50000000,Free,0.00,Everyone,Travel & Local,2-Aug-18,Varies with device,Varies with device
3202,trivago: Hotels & Travel,TRAVEL_AND_LOCAL,4.2,219848,NaN,50000000,Free,0.00,Everyone,Travel & Local,2-Aug-18,Varies with device,Varies with device


Since there are many apps that are repeated, we could keep the first of each duplicate app and remove the rest

In [495]:
apps_df.drop_duplicates(subset ="App",keep = "first", inplace = True) 
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0.0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0.0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000.0,50000000,Free,0.0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000,Free,0.0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53000000.0,5000,Free,0.0,Everyone,Education,25-Jul-17,1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3600000.0,100,Free,0.0,Everyone,Education,6-Jul-18,1,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9500000.0,1000,Free,0.0,Everyone,Medical,20-Jan-17,1,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,19-Jan-15,Varies with device,Varies with device


<h2>Cleaning Data</h2>

We will now try to look for any corrupted/abnormal data and deal with it.

<h3>Category</h3>

We list all of the unique attributes and see if something doesn't fit

In [496]:
apps_df.Category.unique()

array(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY',
       'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION',
       'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
       'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL',
       'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL',
       'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
       'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION'],
      dtype=object)

Everything seems to be in order

<h3>Rating</h3>

In [497]:
print("Min rating = ",apps_df["Rating"].min())
print("Max rating = ",apps_df["Rating"].max())

Min rating =  1.0
Max rating =  5.0


All ratings seem to be within range

<h3>Type</h3>

In [498]:
apps_df.Type.value_counts()

Free    8902
Paid     756
Name: Type, dtype: int64

Nothing wrong here!

<h2>Content Rating</h2>

In [499]:
apps_df["Content Rating"].unique()

array(['Everyone', 'Teen', 'Everyone 10+', 'Mature 17+',
       'Adults only 18+', 'Unrated'], dtype=object)

All is good

<h2>Genres</h2>

In [500]:
apps_df.Genres.unique()

array(['Art & Design', 'Art & Design;Pretend Play',
       'Art & Design;Creativity', 'Art & Design;Action & Adventure',
       'Auto & Vehicles', 'Beauty', 'Books & Reference', 'Business',
       'Comics', 'Comics;Creativity', 'Communication', 'Dating',
       'Education;Education', 'Education', 'Education;Creativity',
       'Education;Music & Video', 'Education;Action & Adventure',
       'Education;Pretend Play', 'Education;Brain Games', 'Entertainment',
       'Entertainment;Music & Video', 'Entertainment;Brain Games',
       'Entertainment;Creativity', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Lifestyle;Pretend Play',
       'Adventure;Action & Adventure', 'Arcade', 'Casual', 'Card',
       'Casual;Pretend Play', 'Action', 'Strategy', 'Puzzle', 'Sports',
       'Music', 'Word', 'Racing', 'Casual;Creativity',
       'Casual;Action & Adventure', 'Simulation', 'Adventure', 'Board',
       'Trivia', 'Role 

Some of the values here seem to be composed of multiple labels, which are separated by a semicolon (ex: 'Sports;Action & Adventure'). We can change divide this column into 2 columns: 'Primary_Genres' & 'Secondary_Genres'

In [501]:
# pd.concat([apps_df, apps_df["Genres"].str.split(';', expand=True)], axis=1)
split = apps_df["Genres"].str.split(';', expand=True)
# split = split.rename(columns={0:"Primary Genres",1:"Secondary Genres"})
split

,0,1
0,Art & Design,None
1,Art & Design,Pretend Play
2,Art & Design,None
3,Art & Design,None
4,Art & Design,Creativity
...,...,...
10836,Education,None
10837,Education,None
10838,Medical,None
10839,Books & Reference,None


In [502]:
apps_df = pd.concat([apps_df,split],axis=1)
apps_df = apps_df.rename(columns={0:"Primary Genres",1:"Secondary Genres"})
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Primary Genres,Secondary Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0.0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up,Art & Design,None
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up,Art & Design,Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0.0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up,Art & Design,None
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000.0,50000000,Free,0.0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up,Art & Design,None
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000,Free,0.0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up,Art & Design,Creativity


<h3>Last Updated</h3>

In [503]:
apps_df["Last Updated"].unique()

array(['7-Jan-18', '15-Jan-18', '1-Aug-18', ..., '20-Jan-14', '16-Feb-14',
       '23-Mar-14'], dtype=object)

Nothing seems wrong here!

<h3>Current Ver</h3>

In [504]:
apps_df["Current Ver"].unique()

array(['1.0.0', '2.0.0', '1.2.4', ..., '1.0.612928', '0.3.4', '2.0.148.0'],
      dtype=object)

Aside from the fact that some values have the vale 'Varies with device', nothing seems to be wrong.

<h3>Android Ver</h3>

In [505]:
apps_df["Android Ver"].unique()

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up',
       'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up',
       '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up',
       '5.1 and up', '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up',
       '3.2 and up', '4.4W and up', '7.1 and up', '7.0 - 7.1.1',
       '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up',
       '4.1 - 7.1.1', nan, '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1',
       '5.0 - 7.1.1'], dtype=object)

Nothing out of the ordinary!

<h2>Finish</h2>

Finally, we save the dataframe into a new csv file

In [507]:
apps_df.to_csv("./data/googleplaystore_clean.csv",index=False)